# ALeRCE classes

https://github.com/ZwickyTransientFacility/ztf-avro-alert

1. **AGN:** Active Galactic Nuclei
1. **Blazar:** Blazar
1. **CV/Nova:** Cataclysmic Variable Star/Nova
1. **Ceph:** Cepheid Variable Star
1. **DSCT:** Delta Scuti Star
1. **EA:** Eclipsing Algol
1. **EB/EW:** Eclipsing Binaries/Eclipsing W Ursa Majoris
1. **LPV:** Long Period Variable
1. **Periodic-Other:** Periodic-Other
1. **QSO:** Quasi-Stellar Object
1. **RRL:** RRLyrae Variable Star
1. **RSCVn:** RS Canum Venaticorum
1. **SLSN:** Super Luminous Supernova
1. **SNII:** Supernova II
1. **SNIIb:** Supernova IIb
1. **SNIIn:** Supernova IIn
1. **SNIa:** Supernova Ia
1. **SNIbc:** Supernova Ibc
1. **TDE:** Tidal disruption event (to remove)
1. **YSO:** Young Stellar Object
1. **ZZ:** ZZ Ceti Stars (to remove)

In [1]:
import sys
sys.path.append('../../')

In [2]:
%load_ext autoreload
%autoreload 2
from vstars.alerce_utils import process_df_labels, process_df_detections
import numpy as np
import pandas as pd

load_rootdir = '/home/opimentel/tesis/surveys_data'
survey_name = 'alerceZTFv7.1'
uses_corr = True # False only with SNe
df_index_names = {
    'oid':'oid', # object id
    'oid_det':'objectId', # object id
    'label':'classALeRCE', # object class name
    'ra':'ra',
    'dec':'dec',
    'band':'fid', # band
    'obs_day':'mjd', # days
    'obs':'magpsf_corr' if uses_corr else 'magpsf', # observations
    'obs_error':'sigmapsf_corr' if uses_corr else 'sigmapsf', # observation errors
}
detections_cols = ['objectId', 'fid', 'mjd', df_index_names['obs'], df_index_names['obs_error']]

### load files and processing
detections_df = pd.read_parquet(f'{load_rootdir}/{survey_name}/detections_with_xmatch')
detections_df, det_objs = process_df_detections(detections_df, df_index_names['oid_det'], df_index_names['oid'], detections_cols)
print(f'detections_df - columns: {list(detections_df.columns)} - id: {detections_df.index.name}')

labels_df = pd.read_csv(f'{load_rootdir}/{survey_name}/dfcrossmatches_prioritized_v7.0.1.csv')
labels_df,_ = process_df_labels(labels_df, df_index_names['oid'], det_objs)
print(f'labels - columns: {list(labels_df.columns)} - id: {labels_df.index.name}')

### print info
classes = set(labels_df[df_index_names['label']].values)
print('classes:', classes)

detections_df - columns: ['fid', 'mjd', 'magpsf_corr', 'sigmapsf_corr'] - id: oid
labels - columns: ['classALeRCE', 'ra', 'dec', 'period', 'source', 'id_source', 'class_source', 'separation_arcsec'] - id: oid
classes: {'DSCT', 'AGN', 'LPV', 'SLSN', 'SNII', 'EA', 'QSO', 'Blazar', 'Ceph', 'SNIIb', 'SNIIn', 'YSO', 'NLQSO', 'TDE', 'NLAGN', 'Periodic-Other', 'ZZ', 'CV/Nova', 'RSCVn', 'RRL', 'EB/EW', 'SNIbc', 'SNIa'}


In [3]:
detections_df.info()
detections_df[-20:]

<class 'pandas.core.frame.DataFrame'>
Index: 3785884 entries, ZTF17aaafyya to ZTF20abcxmfu
Data columns (total 4 columns):
fid              int64
mjd              float64
magpsf_corr      float64
sigmapsf_corr    float64
dtypes: float64(3), int64(1)
memory usage: 144.4+ MB


,fid,mjd,magpsf_corr,sigmapsf_corr
oid,,,,
ZTF20abcxmfu,2,59086.386597,18.835959,0.082238
ZTF20abcxmfu,2,59063.378160,18.837938,0.093653
ZTF20abcxmfu,2,59087.373218,18.735105,0.054769
ZTF20abcxmfu,2,59026.475370,18.599940,0.069426
ZTF20abcxmfu,2,59078.334687,18.791681,0.057921
ZTF20abcxmfu,2,59063.372917,18.729684,0.105329
ZTF20abcxmfu,2,59075.331447,18.804337,0.064222
ZTF20abcxmfu,2,59036.376898,18.645068,0.095453
ZTF20abcxmfu,2,59072.331921,18.731310,0.072716


In [4]:
labels_df.info()
labels_df[:20]

<class 'pandas.core.frame.DataFrame'>
Index: 110043 entries, ZTF19aapcxhy to ZTF18abgqxlw
Data columns (total 8 columns):
classALeRCE          110043 non-null object
ra                   110043 non-null float64
dec                  110043 non-null float64
period               67457 non-null object
source               110043 non-null object
id_source            110043 non-null object
class_source         110043 non-null object
separation_arcsec    110043 non-null float64
dtypes: float64(3), object(5)
memory usage: 7.6+ MB


,classALeRCE,ra,dec,period,source,id_source,class_source,separation_arcsec
oid,,,,,,,,
ZTF19aapcxhy,AGN,154.202129,18.723076,NaN,Oh2015,5.877420127343739e+17,AGN_galaxy_dominated,0.227455
ZTF18abtyspw,AGN,25.660298,0.087434,NaN,Oh2015,5.880155092805878e+17,AGN_galaxy_dominated,0.141792
ZTF18abwtbad,AGN,51.846346,0.739559,NaN,Oh2015,5.877315136939624e+17,AGN_galaxy_dominated,0.084636
ZTF18acvgdfy,AGN,134.407409,5.472596,NaN,Oh2015,5.877327033915148e+17,AGN_galaxy_dominated,0.087190
ZTF18aadyxlg,AGN,125.577004,33.091120,NaN,Oh2015,5.880133827239608e+17,AGN_galaxy_dominated,0.073570
ZTF19aapuscr,AGN,199.102327,-2.090396,NaN,Oh2015,5.87724649802236e+17,AGN_galaxy_dominated,0.038667
ZTF19aanxuxz,AGN,212.636953,-2.822530,NaN,Oh2015,5.877297769066006e+17,AGN_galaxy_dominated,0.181073
ZTF19aaohxwd,AGN,175.347895,21.936834,NaN,Oh2015,5.877420610690746e+17,AGN_galaxy_dominated,0.150095
ZTF18aceqjzp,AGN,139.303760,37.075558,NaN,Oh2015,5.882978641805313e+17,AGN_galaxy_dominated,0.456351


In [5]:
%load_ext autoreload
%autoreload 2
from vstars.alerce_utils import get_valid_classes_objs, filter_by_valid_objs
    
target_classes = [
    #'EA', # Eclipsing Binaries
    'EB/EW', # Eclipsing Binaries
    'Ceph', # Cefeidas
    'RRL', # RR Lyrae
    'DSCT', # Delta Scuti
    'LPV', # Long Period Variables
]
valid_objs = get_valid_classes_objs(labels_df, df_index_names, target_classes)
print(valid_objs)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
['ZTF18abwwdsc', 'ZTF18aaakigd', 'ZTF18aaavkyj', 'ZTF18abwwdxw', 'ZTF19aaocniv', 'ZTF17aacemqz', 'ZTF18aaiyfjx', 'ZTF19aaczymt', 'ZTF18abvpirg', 'ZTF18abccnft', 'ZTF18aazzfhx', 'ZTF18abvtdoa', 'ZTF18absvbuc', 'ZTF18abmxafi', 'ZTF18acpewdw', 'ZTF18abhqdkc', 'ZTF18acrmlma', 'ZTF18accipen', 'ZTF18aagwwiq', 'ZTF17aadfivr', 'ZTF18abmnocu', 'ZTF18abnnbys', 'ZTF17aaadqhs', 'ZTF18abtjmni', 'ZTF18aagrcvb', 'ZTF17aabpwzy', 'ZTF17aacoobq', 'ZTF18abtpdzf', 'ZTF18actwqrc', 'ZTF18abomkrt', 'ZTF18abvfaju', 'ZTF18accfbxq', 'ZTF18abwerjv', 'ZTF18abxeian', 'ZTF18aaadqzu', 'ZTF17aaawdie', 'ZTF18acrulfg', 'ZTF18abvpmxu', 'ZTF18aaaatsz', 'ZTF18acszysu', 'ZTF19aaklsfm', 'ZTF18abgcnrf', 'ZTF18abmouca', 'ZTF18acapcrd', 'ZTF19aaafqwi', 'ZTF17aaadpqo', 'ZTF17aaakjph', 'ZTF18abynvbg', 'ZTF19aajwdvy', 'ZTF17aabureo', 'ZTF18abwklby', 'ZTF17aabulcr', 'ZTF17aaawivn', 'ZTF18acuehfu', 'ZTF18aabgaft', 'ZTF18actzktj', 'ZTF17aacmgja',

In [6]:
new_detections_df = filter_by_valid_objs(detections_df, valid_objs)
print(new_detections_df.info())
new_detections_df[:50]

<class 'pandas.core.frame.DataFrame'>
Index: 2667429 entries, ZTF17aaafyya to ZTF20abceckn
Data columns (total 4 columns):
fid              int64
mjd              float64
magpsf_corr      float64
sigmapsf_corr    float64
dtypes: float64(3), int64(1)
memory usage: 101.8+ MB
None


,fid,mjd,magpsf_corr,sigmapsf_corr
oid,,,,
ZTF17aaafyya,1,58791.283368,15.262989,0.004987
ZTF17aaafyya,1,59081.470544,15.281165,0.007919
ZTF17aaafyya,1,59067.403970,15.245437,0.000493
ZTF17aaafyya,1,58793.249965,15.205705,0.010043
ZTF17aaafyya,1,58793.239028,15.228786,0.009911
ZTF17aaafyya,1,58747.324676,15.226640,0.009151
ZTF17aaafyya,1,58334.472708,15.233562,0.009774
ZTF17aaafyya,1,58677.470833,15.276610,0.009193
ZTF17aaafyya,1,58332.463079,15.212075,0.003076


In [7]:
new_labels_df = filter_by_valid_objs(labels_df, valid_objs)
print(new_labels_df.info())
new_labels_df[:50]

<class 'pandas.core.frame.DataFrame'>
Index: 82237 entries, ZTF18abwwdsc to ZTF18abgqxlw
Data columns (total 8 columns):
classALeRCE          82237 non-null object
ra                   82237 non-null float64
dec                  82237 non-null float64
period               59854 non-null object
source               82237 non-null object
id_source            82237 non-null object
class_source         82237 non-null object
separation_arcsec    82237 non-null float64
dtypes: float64(3), object(5)
memory usage: 5.6+ MB
None


,classALeRCE,ra,dec,period,source,id_source,class_source,separation_arcsec
oid,,,,,,,,
ZTF18abwwdsc,Ceph,326.755845,-8.060673,2.18895,CRTSnorth,1007116010622.0,Cep-II,0.676851
ZTF18aaakigd,Ceph,65.440016,34.069809,2.11360,CRTSnorth,1135020009815.0,ACEP,0.390734
ZTF18aaavkyj,Ceph,192.885044,24.122642,1.09572,CRTSnorth,1123064031941.0,ACEP,0.291110
ZTF18abwwdxw,Ceph,324.865973,-17.296145,1.11714,CRTSnorth,1018112055304.0,ACEP,1.881608
ZTF19aaocniv,Ceph,248.580619,-16.015739,1.3049582,CRTSnorth,1015086048064.0,Cep-II,0.405458
ZTF17aacemqz,Ceph,117.977671,27.561363,16.2488,CRTSnorth,1126038065052.0,Cep-II,1.092943
ZTF18aaiyfjx,Ceph,245.464370,38.909590,1.04986,CRTSnorth,1138071060852.0,Cep-II,0.610218
ZTF19aaczymt,Ceph,228.810756,-14.901231,4.3680244,CRTSnorth,1015079060794.0,Cep-II,0.422664
ZTF18abvpirg,Ceph,351.955300,-9.388094,119.947,CRTSnorth,1009125035570.0,Cep-II,0.266861


In [8]:
print(new_labels_df.loc['ZTF18aaavkyj'])
print(new_detections_df.loc['ZTF18aaavkyj'])

classALeRCE                     Ceph
ra                           192.885
dec                          24.1226
period                       1.09572
source                     CRTSnorth
id_source            1123064031941.0
class_source                    ACEP
separation_arcsec            0.29111
Name: ZTF18aaavkyj, dtype: object
              fid           mjd  magpsf_corr  sigmapsf_corr
oid                                                        
ZTF18aaavkyj    1  58567.446736    15.051076       0.014877
ZTF18aaavkyj    1  58588.213599    14.874550       0.024162
ZTF18aaavkyj    2  58589.241782    14.827786       0.011289


In [9]:
import pandas as pd

### save files
save_root_dir = f'../../data/{survey_name}'
new_labels_df.to_parquet(f'{save_root_dir}/labels_vs.parquet')
new_detections_df.to_parquet(f'{save_root_dir}/detections_vs.parquet')